In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
import src.config as config
from src.inference import (
    get_feature_store
)

from datetime import datetime, timezone, timedelta

def get_current_utc_hour_ceiled():
    """Get current UTC time ceiled to the next hour"""
    current = datetime.now(timezone.utc)
    if current.minute > 0 or current.second > 0:
        # Ceil to next hour
        current = current + timedelta(hours=1)

    # Reset minutes, seconds, and microseconds
    current = current.replace(minute=0, second=0, microsecond=0)
    return current

# Usage
current_hour = get_current_utc_hour_ceiled()
print(f"Current UTC hour (ceiled): {current_hour}")

Current UTC hour (ceiled): 2025-03-03 08:00:00+00:00


In [4]:
def fetch_predictions():
    current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour == current_hour)

    return query.read()

In [5]:
fs = get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1
)

query = fg.select_all()

2025-03-03 02:17:15,208 INFO: Initializing external client
2025-03-03 02:17:15,209 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 02:17:16,096 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215651


In [6]:
df= query.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.78s) 


In [7]:
df

,pickup_location_id,predicted_demand,pickup_hour
0,254,0.0,2025-03-03 06:00:00+00:00
1,68,129.0,2025-03-03 06:00:00+00:00
2,207,0.0,2025-03-03 06:00:00+00:00
3,202,0.0,2025-03-03 06:00:00+00:00
4,91,0.0,2025-03-03 06:00:00+00:00
...,...,...,...
748,125,24.0,2025-03-03 08:00:00+00:00
749,173,0.0,2025-03-03 08:00:00+00:00
750,218,1.0,2025-03-03 08:00:00+00:00
751,72,1.0,2025-03-03 08:00:00+00:00


In [8]:
a.info()

NameError: name 'a' is not defined

In [9]:
from datetime import datetime, timezone, timedelta

def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    # Extract components from the rounded-up time
    year = next_hour.year
    month = next_hour.month
    day = next_hour.day
    hour = next_hour.hour

    # Create date strings in YYYY-MM-DD format
    current_date = f"{year}-{month:02d}-{day:02d}"
    next_date = next_hour + timedelta(days=1)
    next_date_str = next_date.strftime("%Y-%m-%d")

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    # # First get the full day's data from Hopsworks
    # df = fg.filter(
    #     (fg.pickup_hour >= current_date) &
    #     (fg.pickup_hour < next_date_str)
    # ).read()

    # Then filter for next hour in the DataFrame
    hour_str = f"{current_date} {hour:02d}:00"
    df_hour = df[df['pickup_hour'] == hour_str]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Querying for date range: {current_date} to {next_date_str}")
    print(f"Filtering for hour: {hour_str}")
    print(f"Found {len(df_hour)} records")
    return df_hour

# Example usage:
predictions = fetch_next_hour_predictions()

2025-03-03 02:17:37,472 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 02:17:37,474 INFO: Initializing external client
2025-03-03 02:17:37,475 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 02:17:38,253 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215651
Current UTC time: 2025-03-03 07:17:37.472382+00:00
Next hour: 2025-03-03 08:00:00+00:00
Querying for date range: 2025-03-03 to 2025-03-04
Filtering for hour: 2025-03-03 08:00
Found 251 records


In [10]:
now = datetime.now(timezone.utc)

In [11]:
predictions

,pickup_location_id,predicted_demand,pickup_hour
502,46,0.0,2025-03-03 08:00:00+00:00
503,162,119.0,2025-03-03 08:00:00+00:00
504,261,15.0,2025-03-03 08:00:00+00:00
505,190,0.0,2025-03-03 08:00:00+00:00
506,18,0.0,2025-03-03 08:00:00+00:00
...,...,...,...
748,125,24.0,2025-03-03 08:00:00+00:00
749,173,0.0,2025-03-03 08:00:00+00:00
750,218,1.0,2025-03-03 08:00:00+00:00
751,72,1.0,2025-03-03 08:00:00+00:00


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 753 entries, 0 to 752
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  753 non-null    int32                  
 1   predicted_demand    753 non-null    float64                
 2   pickup_hour         753 non-null    datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float64(1), int32(1)
memory usage: 14.8 KB


In [13]:
import pandas as pd
current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

In [14]:
dt = current_date.ceil('h')


In [15]:
fs = get_feature_store()  
fg = fs.get_feature_group(  
    name=config.FEATURE_GROUP_MODEL_PREDICTION,  
    version=1  
)  
query = fg.select_all() 
# query = query.filter(fg.pickup_hour >= "2025-02-13 04:00")
query = query.filter(fg.pickup_hour > "2025-02-13 05:00:00")  
results = query.read()

2025-03-03 02:17:53,669 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 02:17:53,671 INFO: Initializing external client
2025-03-03 02:17:53,671 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 02:17:54,462 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215651
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.47s) 


In [16]:
fg.filter((fg.pickup_hour >= "2025-02-13") & (fg.pickup_hour < "2025-02-14")).read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.42s) 


,pickup_location_id,predicted_demand,pickup_hour


In [17]:
results

,pickup_location_id,predicted_demand,pickup_hour
0,254,0.0,2025-03-03 06:00:00+00:00
1,68,129.0,2025-03-03 06:00:00+00:00
2,207,0.0,2025-03-03 06:00:00+00:00
3,202,0.0,2025-03-03 06:00:00+00:00
4,91,0.0,2025-03-03 06:00:00+00:00
...,...,...,...
748,125,24.0,2025-03-03 08:00:00+00:00
749,173,0.0,2025-03-03 08:00:00+00:00
750,218,1.0,2025-03-03 08:00:00+00:00
751,72,1.0,2025-03-03 08:00:00+00:00


In [19]:
def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    fs = get_feature_store()
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

    # Then filter for next hour in the DataFrame
    df_hour = df[df["pickup_hour"] == next_hour]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Found {len(df_hour)} records")
    return df_hour

In [20]:
predictions = fetch_next_hour_predictions()

2025-03-03 02:18:13,687 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 02:18:13,689 INFO: Initializing external client
2025-03-03 02:18:13,690 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 02:18:14,460 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215651
Current UTC time: 2025-03-03 07:18:13.687047+00:00
Next hour: 2025-03-03 08:00:00+00:00
Found 251 records


In [21]:
fs = get_feature_store()
fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)



2025-03-03 02:18:19,690 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 02:18:19,692 INFO: Initializing external client
2025-03-03 02:18:19,693 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 02:18:20,414 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215651


In [22]:
df = fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.81s) 


In [23]:
df["pickup_hour"].max()

Timestamp('2025-03-03 08:00:00+0000', tz='Etc/UTC')

In [24]:
now = datetime.now(timezone.utc)
next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
print(next_hour)

2025-03-03 08:00:00+00:00


In [25]:
df[df["pickup_hour"] == next_hour]

,pickup_location_id,predicted_demand,pickup_hour
502,46,0.0,2025-03-03 08:00:00+00:00
503,162,119.0,2025-03-03 08:00:00+00:00
504,261,15.0,2025-03-03 08:00:00+00:00
505,190,0.0,2025-03-03 08:00:00+00:00
506,18,0.0,2025-03-03 08:00:00+00:00
...,...,...,...
748,125,24.0,2025-03-03 08:00:00+00:00
749,173,0.0,2025-03-03 08:00:00+00:00
750,218,1.0,2025-03-03 08:00:00+00:00
751,72,1.0,2025-03-03 08:00:00+00:00
